In [33]:
import os
import sys
import re
import pandas as pd
import numpy as np
from enum import Enum
from typing import Optional, Union, List, Callable
from dataclasses import dataclass, make_dataclass, field

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [27]:
@dataclass
class ValueBase:
    
    buffer: bytearray
    pos: int
    length: int
    signed: bool=False
    order: str='little'
    skiphead: int=0
    skiptail: int=0
    
    @staticmethod
    def bytes2Num(barray, signed:bool=False, order:str='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num, length, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    
    def get_bytes(self):
        return self.buffer[self.pos+self.skiphead
                          :self.pos+self.skiphead-self.skiptail]
        
    def get_value(self):
        return self.bytes2Num(self.get_bytes(), self.signed, self.order)
    
    def get_string(self):
        s = self.get_bytes()
        p = np.argmax(np.logical_and(np.array(s) > 127
                                     ,np.array(s) < 32))
        return str(s[:p], encoding='latin')
    
    def set_value(self, value:Union[int, Enum]):
        if hasattr(value, 'value'):
            value = value.value
        self.buffer[self.pos+self.skiphead
                   :self.pos+self.skiphead-self.skiptail] = self.num2Bytes(value, self.length, self.signed, self.order) 
            
    def inc_value(self, inc_value:int=1):
        self.set_value(self.get_value() + inc_value)
    

In [34]:
def make_names_old(name:str, df:pd.DataFrame):
    '''
    Construct dataclass or nested dataclass from a dataframe
    '''
    if df.shape[1] == 2:
        data = [(_[1], str, field(default=_[0])) for _ in df.values]
        return make_dataclass(name, data)
    else:
        group = df.groupby(df.columns[0])
        data = [(k, dataclass, field(default=make_names_old(k, v.drop(v.columns[0], axis=1)))) for k, v in group]
        return make_dataclass(name, data)

In [ ]:
def make_names_(name:str, df:pd.DataFrame):
    '''
    Construct dataclass or nested dataclass from a dataframe
    '''
    if df.shape[1] == 2:
        data = [(_[1], str, field(default=_[0])) for _ in df.values]
        return make_dataclass(name, data)
    else:
        group = df.groupby(df.columns[0])
        if group.ngroups == 1:
            return make_names_(name, df.drop(df.columns[0], axis=1))
        data = [(k, dataclass, field(default=make_names_(k, v.drop(v.columns[0], axis=1)))) for k, v in group]
        return make_dataclass(name, data)
    
def make_names(name:str, df:pd.DataFrame, ignore_cols:List=[]):
    return make_names_(name=name, df=df[[_ for _ in df.columns if not _ in ignore_cols]])

In [ ]:
def clean_duplicates(df:pd.DataFrame, groupby:Optional[Union[List, str]]=None):
    if groupby is None:
        col = df.columns[-1]
        df = df.groupby(col).apply(lambda x: x.reset_index())
        if df.index.nlevels > 1:
            df = df.droplevel(0)
        df[col] = df.apply(lambda x: f'{re.sub("_+", "_", x[col])}{("_" + str(x.name)) if x.name > 0 else ""}', axis=1)
        return df.reset_index(drop=True).drop('index', axis=1)
    else:
        return df.groupby(groupby).apply(clean_duplicates).reset_index(drop=True)